In [1]:
import importlib
import sys
import h5py
import scipy.io
import csv
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.linalg import logm,expm, sqrtm
from scipy.stats import zscore
from scipy.stats import mode
from sklearn.decomposition import PCA, FastICA
from sklearn.metrics import pairwise_distances
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LogisticRegression

import torch
import torch.nn as nn
import torch.optim as optim


sys.path.insert(0, "D:/PhD/tangent_space_analysis/")
                
import TSAfuncs as tsa
importlib.reload(tsa)

c:\Users\davcu\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\davcu\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\davcu\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\Users\davcu\AppData\Local\Temp\ipykernel_10808\1022575765.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arro

<module 'TSAfuncs' from 'D:\\PhD/tangent_space_analysis\\TSAfuncs.py'>

In [2]:
ROI = scipy.io.loadmat('filteredROI.mat')
ROI = ROI['ROItokeep']
nparcels = [100, 400, 1000]
ROI_to_keep = dict()
for i,n in enumerate(nparcels):
    ROI_to_keep[n] = ROI[0][i][0]


In [3]:


parent_dir = os.getcwd()

#parent_dir = 'parent dir here'
dfOG = pd.read_csv(parent_dir + "\\" + 'allRecordings_dataframe.csv')

dfOG = dfOG[dfOG['numparcels'] == 100]
# dfOG = dfOG[(dfOG['Num Subnets'] == 17) & (dfOG['Num Parcels'] == 300)]
# tasklist = ['Rest', 'Motor']#, 'Memory']
FCs = list()

# rest_df = pd.DataFrame()
# dfOG.head(10)

for index,row in dfOG.iterrows():
    filename = row['filepath'] +'/' + 'FCs/' +'FC_' + row['subject'] + '_recid=' + str(row['recid']) + '.mat' #+ row['filename']
    filename = filename.replace( '/scratch2/DavorCuric/CANBIND/' ,"D:/PhD/CANBIND/")
    
    try:
        f = scipy.io.loadmat(filename)
        subject_id = str(row['subject'])
        state = int(row['controls'])*'controls' + int(row['nonresponders'])*'nonresponders' + int(row['responders'])*'responders'
        
        session = row['session']

        if state == 'controls':
            continue
        if state == '':
            continue

        FCs.append(tsa.FC(np.array(f['FC']), subject = subject_id, state = state, session = session))
    except:
        print('file not found: ' + filename)
    

file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01CAM0005_recid=710.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01MCU0011_recid=729.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01MCU0055_recid=757.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01TGH0002_recid=794.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01TGH0077_recid=825.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01TGH0078_recid=826.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01TGH0080_recid=828.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01UBC0049_recid=859.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01UBC0067_recid=876.mat
file not found: D:/PhD/CANBIND/CANBIND_WK2_100_PARCELS/FCs/FC_CBN01UCA0024_recid=893.mat
file not found: D:/PhD/CANBIND/CANBIND_WK8_100_PARCELS/FCs/FC_CBN01CAM0005_recid=1244.mat
file not found: D:/P

In [30]:
#rest retest with identity reference

regvals = np.array([1])#0.01, 0.1, 1, 10 ])
trt = tsa.test_retest(FCs)

for r in regvals:
    score = trt.ts_test_retest(np.array([r]))#, refinv = 'logm')
    score_classic = trt.classic_test_retest()
    print('tangent space score = ' + str(score) + ', non-tangent space score = ' + str(score_classic))


tangent space score = 0.930894308943087, non-tangent space score = 0.861788617886177


In [ ]:
sys.path.insert(0, "D:/PhD/tangent_space_analysis/")
                
import TSAfuncs as tsa
importlib.reload(tsa)

#test retest with non-identity reference
regvals = np.array([.01])#0.01, 0.1, 1, 10 ])
trt = tsa.test_retest(FCs)


#test with individual reg value
for r in regvals:
    score = trt.ts_test_retest(
        regvals = np.array([r]), 
        refinv = 'logm'
        )
    
    #subject_score_classic, state_score_classic, conditional_score_classic = trt.classic_test_retest()
    #print('ts subject score = ' + str(subject_score) + ', non-ts subject score = ' + str(subject_score_classic))
    #print('ts state score = ' + str(state_score) + ', non-ts state score = ' + str(state_score_classic))
